In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
# --------------------------------------------------------------------------
# | БЛОК 1: Настройка файлов и весов                                       |
# --------------------------------------------------------------------------

# Укажите пути к вашим файлам с предсказаниями
file_1_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_34.csv'
file_2_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_37.csv'

# Укажите скоры ваших моделей (они будут использоваться как веса)
# Модели с более высоким скором будут иметь большее влияние
weight_1 = 0.50004  # Вес для file_1
weight_2 = 0.49912  # Вес для file_2

# Путь для сохранения итогового файла
output_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_38.csv'

print("--- Настройки ---")
print(f"Файл 1: {file_1_path} (вес: {weight_1})")
print(f"Файл 2: {file_2_path} (вес: {weight_2})")
print(f"Итоговый файл: {output_path}\n")


--- Настройки ---
Файл 1: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_34.csv (вес: 0.50004)
Файл 2: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_37.csv (вес: 0.49912)
Итоговый файл: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_38.csv


In [4]:
# --------------------------------------------------------------------------
# | БЛОК 2: Загрузка и подготовка данных                                   |
# --------------------------------------------------------------------------

print("Шаг 1: Загрузка файлов с предсказаниями...")
df1 = pd.read_csv(file_1_path)
df2 = pd.read_csv(file_2_path)

Шаг 1: Загрузка файлов с предсказаниями...


In [5]:
# --------------------------------------------------------------------------
# | БЛОК 3: Преобразование рангов в оценки (Scores)                        |
# --------------------------------------------------------------------------

# Вспомогательная функция для преобразования рангов в оценки
# Чем ниже ранг (т.е. лучше), тем выше будет оценка
def rank_to_score(series):
    # n - количество элементов в группе
    n = len(series)
    # Формула: score = 1 - (rank - 1) / n
    # Ранг 1 -> score 1.0
    # Ранг n -> score 1/n
    return 1.0 - (series - 1) / n

print("Шаг 2: Преобразование рангов в оценки для каждой группы...")

# Применяем функцию к каждой группе ranker_id
# .transform() позволяет применить функцию к группе и вернуть результат того же размера, что и исходный DataFrame
df1['score_1'] = df1.groupby('ranker_id')['selected'].transform(rank_to_score)
df2['score_2'] = df2.groupby('ranker_id')['selected'].transform(rank_to_score)

print("Преобразование завершено.\n")

Шаг 2: Преобразование рангов в оценки для каждой группы...
Преобразование завершено.


In [6]:
# --------------------------------------------------------------------------
# | БЛОК 4: Объединение и взвешенное усреднение                             |
# --------------------------------------------------------------------------

print("Шаг 3: Объединение оценок и взвешенное усреднение...")

# Объединяем два DataFrame по 'Id', чтобы для каждой строки были обе оценки
# Убираем лишние колонки, чтобы не было дубликатов
merged_df = pd.merge(
    df1[['Id', 'ranker_id', 'score_1']],
    df2[['Id', 'score_2']],
    on='Id'
)

# Вычисляем взвешенную среднюю оценку
merged_df['blended_score'] = (
    merged_df['score_1'] * weight_1 +
    merged_df['score_2'] * weight_2
) / (weight_1 + weight_2)

print("Усреднение завершено.\n")

Шаг 3: Объединение оценок и взвешенное усреднение...
Усреднение завершено.


In [8]:
# --------------------------------------------------------------------------
# | БЛОК 5: Преобразование итоговых оценок в ранги и сохранение              |
# --------------------------------------------------------------------------

print("Шаг 4: Преобразование итоговых оценок обратно в ранги...")

# Группируем по ranker_id и ранжируем по новой усредненной оценке
# ascending=False, так как более высокая оценка означает лучший ранг (меньшее число)
# method='first' гарантирует, что не будет одинаковых рангов в одной группе
merged_df['selected'] = merged_df.groupby('ranker_id')['blended_score'].transform(
    lambda s: s.rank(method='first', ascending=False)
).astype(int)

# Создаем финальный DataFrame в нужном формате
submission_df = merged_df[['Id', 'ranker_id', 'selected']]

print("Ранжирование завершено.\n")

# Сохраняем результат
print(f"Шаг 5: Сохранение итогового файла в {output_path}...")
submission_df.to_csv(output_path, index=False)

print("\n--- Готово! ---")
print("Пример содержимого итогового файла:")
print(submission_df.head())

Шаг 4: Преобразование итоговых оценок обратно в ранги...
Ранжирование завершено.

Шаг 5: Сохранение итогового файла в C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_38.csv...

--- Готово! ---
Пример содержимого итогового файла:
         Id                         ranker_id  selected
0  18144679  c9373e5f772e43d593dd6ad2fa90f67a         8
1  18144680  c9373e5f772e43d593dd6ad2fa90f67a        51
2  18144681  c9373e5f772e43d593dd6ad2fa90f67a       244
3  18144682  c9373e5f772e43d593dd6ad2fa90f67a        70
4  18144683  c9373e5f772e43d593dd6ad2fa90f67a        87
